# Inpainting de Objetos Largos

### Projeto Final de Processamento de Imagens (SCC-0251)
#### Instituto de Ciências Matemáticas e de Computação - 2018.1
#### Universidade de São Paulo - São Carlos/SP

## Integrantes
* Gabriel Romualdo Silveira Pupo (Nº USP 9896250)
* Matheus Carvalho Raimundo (Nº USP 10369014)

## Índice Referencial
1. Objetivo
2. Conjunto de Imagens
3. Organização do Repositório
4. Conceitos e Introdução Teórica
5. Algoritmo Parcial
6. Algoritmo Final
7. Detalhes de Codificação
8. Resultados
9. Conclusões
10. Referências

## 1. Objetivo
Neste projeto, foi implementado um algoritmo avançado de Inpainting, capaz de remover objetos de grande dimensão e formato de uma imagem colorida, reconstruíndo-a. O domínio de espaço por trás deste objeto foi reconstruído através de outras regiões da imagem próximas ao objeto removido ou de semelhança com as extremidades deste, arranjando os blocos remendados de uma forma visualmente aceitável.

## 2. Conjunto de Imagens
Foram usadas apenas imagens sem restrições de direitos autorais para construir nosso conjunto de teste. Algumas dessas imagens passaram por edições (softwares terceiros) de forma a inserir um objeto a ser removido (danificar a entrada) ou construir uma máscara. Todas as informações sobre as imagens se encontram no diretório [_ImagesAndAttribution_](ImagesAndAttribution).

## 3. Organização do Repositório
O repositório foi organizado da seguinte maneira:
* [ImagesAndAttribution](ImagesAndAttribution): aqui se encontram as imagens que foram usadas como teste no projeto. O underscore (\_) determina diferentes variações de uma mesma imagem, como diferentes resoluções por exemplo.
* [FirstsResults](FirstsResults): aqui se encontram os resultados iniciais (parciais) obtidos nas primeiras versões do algoritmo. Para o relatório final do trabalho, desconsiderar tal diretório.
* [Results](Results): aqui se encontram os resultados obtidos nos casos de teste construídos.
* [TestCases](TestCases): casos de teste usados para executar o algoritmo.
* [InpaintingLargeObjects.py](InpaintingLargeObjects.py): Código em Python. Executado com Python 3; pode-se utilizar os casos de teste disponíveis em [_TestCases_](TestCases).
* [ProjectReport.ipynb](ProjectReport.ipynb): Relatório final do projeto (este relatório).

## 4. Conceitos e Introdução Teórica
__Inpainting__ é um processo de reconstrução de partes de imagens. O processo de reconstrução pode ser feito de diversas maneiras: análise de texturas, onion ring, FFTs, etc.. Neste projeto, será utilizado o algoritmo onion ring, para reconstruir a imagem de fora para dentro. Alguns conceitos frequentes são utilizados, sendo que entre eles, os de maior relevância são:
* __Patch ou Bloco__: é uma região ou janela onde o Inpainting será feito. Pode ser uma única grande região (algoritmo parcial) ou inúmeras regiões de pequeno tamanho pré-definido (algoritmo final).
* __Janela__: uma parte/pedaço (slicing) da imagem.
* __Thread__: processo leve, usado para processamento paralelo.

## 5. Algoritmo Parcial
O algoritmo parcial (relatório parcial) já era capaz de realizar uma busca e uma substituição na imagem em escala de cinzas. Ele recebe uma imagem de entrada e uma imagem de máscara que determina uma única região onde o Inpainting pode ser realizado. Também recebe o nome de um arquivo para salvar como resultado do processamento. Em alto nível, o algoritmo pode ser resumido da seguinte maneira:
    1. Obter um único patch para realizar preenchimento: obter os índices x e y na imagem através da máscara.
    2. Buscar por regiões da imagem que podem ser usadas como substituição para este patch: analisar e comparar com uma janela do que há em volta desse patch (bordas). Obter região por analise pixel a pixel, de forma a obter a mais semelhante.
    3. Quando encontrada a região, realizar a substituição do conteúdo (Inpainting).
    4. Salvar resultado final.

Este é um algoritmo extremamente ineficiente, porque ele analisa, para um único patch, todo o restante da imagem. Isto não é necessário, porque sabe-se que analisando apenas o que há próximo deste patch já é o bastante para determinar o que será usado para realizar o Inpainting. Além disso, ocasionalmente, patches muito grandes não recebem um bom preenchimento, pois não há região de tal tamanho e tal semelhança na imagem, o que entra em conflito com o objetivo principal do trabalho. Por fim, o algoritmo não trabalha com imagens coloridas. Como conclusões têm-se:
* O algoritmo tem alta complexidade e demanda um longo tempo para execução;
* O algoritmo consegue realizar Inpainting apenas de um patch por vez;
* O algoritmo não é capaz de produzir um bom resultado com máscaras grandes;
* O algoritmo trabalha apenas com imagens em escala de cinzas.

## 6. Algoritmo Final
A solução para os problemas do algoritmo parcial se encontram implementadas no Algoritmo Final. Além de receber a imagem de entrada, uma imagem de máscara e o nome de um arquivo para salvar como resultado do processamento, o algoritmo final também recebe dois importantes parâmetros do usuário: o _tamanho de textura_ e a _razão de janela de análise_.

#### Parâmetro: Imagem de Entrada
A imagem inicial onde o Inpainting será realizado. Pode ser uma imagem colorida ou em escala de cinzas.
#### Parâmetro: Imagem de Máscara
A imagem que representa onde o Inpainting será realizado (o que será removido da imagem original). A máscara pode ter mais de um patch (bloco para remoção) e um patch pode adquirir inúmeras formas. Contudo, a máscara deve ter o mesmo tamanho e modo de cor da imagem de entrada: ou as duas são coloridas, ou as duas são em escala de cinza. Na máscara, qualquer valor maior que 0 (em qualquer canal) será o bastante para considerar tal pixel parte da máscara para Inpainting.
#### Parâmetro: Tamanho de Textura
Este é um parâmetro muito importante que é determinante para produzir o resultado final. Para realizar o Inpainting, todos os patches da máscara serão subdivididos em patches menores. Basicamente, esse parâmetro determina o tamanho desses patches menores. Esse valor depende das dimensões da imagem e dos contornos em volta dos objetos em que deseja-se realizar Inpainting. Para uma imagem de dimensão fixa, quanto menor esse valor, maior a quantidade de detalhes usados no preenchimento, e quanto maior esse valor, menor a quantidade de detalhes. Contudo, este valor varia muito de imagem para imagem: às vezes muitos detalhes é algo bom e às vezes é algo ruim. Cabe ao usuário determinar com base na análise da imagem este valor, ou com base na "tentativa e erro" ao executar o algoritmo.
#### Parâmetro: Razão de Janela de Análise
Este parâmetro determina o tamanho da janela que será usada para comparação de um patch. O valor 2 indica que a janela de análise (centrada no patch em que se deseja realizar Inpainting) terá 2 vezes a dimensão desse patch, por exemplo.

### Implementação
O algoritmo realiza o Inpainting de um patch grande da imagem subdividindo-o em patches menores e preenchendo estes de fora para dentro. Em alto nível, este pode ser resumido da seguinte maneira:
    1. Obter todos os primeiros patches menores (mais próximos da extremidade) onde o Inpainting será realizado.
    2. Para cada um desses patches:
        a. Buscar região de maior semelhança na janela de análise, pixel a pixel.
        b. Se encontrou, realiza o Inpainting (substituição bruta de conteúdo) na imagem original e atualiza a máscara de forma a retirar dela este patch, já que este já foi processado.
    3. Repete os passos 1 e 2 até que não existam mais patches na máscara.
    4. Quando concluído, salvar o resultado final.

Como consequências da implementação do algoritmo final, têm-se:
* Menor complexidade (detalhado abaixo);
* Mais de um patch pode ser utilizado ao mesmo tempo, inclusive de formas variadas;
* Bons resultados com máscaras grandes, desde que o tamanho de textura e a razão de janela de análise sejam bem definidos;
* Manipulação de imagens coloridas e em escala de cinzas;
* Como desvantagem, há a necessidade de determinar dois parâmetros a mais.

### Complexidade
A complexidade deste algoritmo é determinada por 3 parâmetros: a máscara de entrada, o tamanho de textura e a razão de janela de análise. Quanto maior é a máscara de entrada, mais patches serão processados. Quanto menor o tamanho de textura, mais patches serão processados, pois maior é a subdivisão destes. Quanto maior a razão de janela de análide, mais pixels serão analisados para cada patch processado. Apesar disso tudo, o parâmetro mais significativo quando se trata de complexidade é a razão da janela de análise, pois quando diminuimos a janela em que os patches trabalham, menos pixels serão analisados para cada um deles, e considerando que é uma análise de ordem quadrática, a complexidade diminui drásticamente. Em nossos testes, em nenhum momento o algoritmo executou por mais de 1 minuto.

#### Processamento Paralelo
A análise dos patches de uma mesma iteração é uma operação que pode ser realizada de forma paralela, pois o resultado de uma análise não deve influenciar em outra, além de que apenas acessos são feitos inicialmente. Sendo assim, foi optado por paralelizar (multi-threading) esta análise, de forma a diminuir o tempo de processamento de uma imagem. Mas para substituição (o Inpainting, propriamente dito), é necessário utilizar um semáfaro de forma que apenas uma Thread modifique a imagem ao mesmo tempo (tornando o Inpainting thread-safe).

## 7. Detalhes de Codificação
Toda a implementação do algoritmo foi feita em Python utilizando as seguintes bibliotecas:
* __ImageIO__: Leitura e escrita dos arquivos de imagens;
* __Numpy__: manipulação das imagens;
* __threading__: processamento paralelo e semáfaros;
* __sys__: nada significante.

Para encontrar os patches a serem preenchidos de fora para dentro na máscara, é usado o método argwhere do Numpy. Com ele, obtém-se os índices x e y de todos os pontos na máscara marcados para remoção:

In [ ]:
np.argwhere(inpaintingMask != 0)

Desses pontos, considera-se apenas aqueles que encontram-se nas extremidades. Para isso, é feita uma verificação para cada ponto encontrado no passo anterior se ele possui algum vizinho que não é parte da máscara:

In [ ]:
if (point[0] - 1 >= 0 and inpaintingMask[point[0] - 1, point[1]].all() == 0 or
    point[1] - 1 >= 0 and inpaintingMask[point[0], point[1] - 1].all() == 0 or
    point[0] - 1 >= 0 and point[1] - 1 >= 0 and inpaintingMask[point[0] - 1, point[1] - 1].all() == 0 or
    point[0] + 1 < inpaintingMask.shape[0] and inpaintingMask[point[0] + 1, point[1]].all() == 0 or
    point[1] + 1 < inpaintingMask.shape[1] and inpaintingMask[point[0], point[1] + 1].all() == 0 or
    point[0] + 1 < inpaintingMask.shape[0] and point[1] + 1 < inpaintingMask.shape[1] and inpaintingMask[point[0] + 1, point[1] + 1].all() == 0 or
    point[0] - 1 >= 0 and point[1] + 1 < inpaintingMask.shape[1] and inpaintingMask[point[0] - 1, point[1] + 1].all() == 0 or
    point[0] + 1 < inpaintingMask.shape[0] and point[1] - 1 >= 0 and inpaintingMask[point[0] + 1, point[1] - 1].all() == 0)

Essa enorme condição poderia ser substituída por uma convolução de matriz __[ [1, 1, 1], [1, 0, 1], [1, 1, 1] ]__, mas não foi feito por motivos de eficiência. Por fim, desconsidera-se alguns pontos que fazem parte de um mesmo patch (não há necessidade de processar um mesmo patch mais de uma vez). Para cada ponto ainda restante, é feito algumas checagens de duplicados:

In [ ]:
if inpaintingMaskPoints[point[0], point[1]] == 0:
        continue
    inpaintingMaskPoints[point[0], point[1]] = 1
    for i in range(textureSize):
        inpaintingMaskPoints[(point[0] - i - 1, point[1])] = 0
        inpaintingMaskPoints[(point[0] + i + 1, point[1])] = 0
        inpaintingMaskPoints[(point[0], point[1] - i - 1)] = 0
        inpaintingMaskPoints[(point[0], point[1] + i + 1)] = 0
        for j in range(textureSize):
            inpaintingMaskPoints[(point[0] - i - 1, point[1] + j + 1)] = 0
            inpaintingMaskPoints[(point[0] + i + 1, point[1] + j + 1)] = 0
            inpaintingMaskPoints[(point[0] - i - 1, point[1] - j - 1)] = 0
            inpaintingMaskPoints[(point[0] + i + 1, point[1] - j - 1)] = 0

Finalmente, o resultado final contendo apenas os pontos que representam um patch é obtido e o algoritmo continua. Agora, as Threads para cada patch são criadas e colocadas para execução na Thread principal (_main_):

In [ ]:
processingThreads = [ ]
for i in patches:
    processingThreads.append(Thread(target=computePatchPoint, args=(originalImage, maskImage, textureSize, analysisWindowSize * 2, i, counter)))
for thread in processingThreads:
    thread.start()
for thread in processingThreads:
    thread.join()

Após execução de todas as Threads, o algoritmo passa para a próxima iteração: adentra a máscara. Ao final, este salva a imagem em disco.

## 8. Resultados
Foram obtidos ótimos resultados com o algoritmo final. Abaixo se encontram alguns deles.

### Krefeld (caso 1)
![Krefeld](ImagesAndAttribution/Krefeld_i.png)
Krefeld é uma cidade da Alemanha _(a mesma que não foi pra final da Copa do Mundo de 2018)_. Aplicando Inpainting nesta imagem, o resultado gerado foi este:
![Krefeld](Results/Krefeld_output.png)
Este não foi um resultado ideal. Porém a imagem foi usada para demonstrar funcionamento em uma resolução 4K. Visualmente, aparenta uma construção demolida.

### Ocean (caso 2)
![Ocean](ImagesAndAttribution/Ocean_480_i.png)
Ocean é uma fotografia de uma praia em Portugal. Aplicando Inpainting sob esta imagem, o resultado gerado foi este:
![Ocean](Results/Ocean_480_output.png)
Este resultado, por outro lado, está ideal. A placa está flutuando, como desejado, e a outra desapareceu.

### Street (caso 3)
![Street](ImagesAndAttribution/Street_480_i.png)
O algoritmo também foi aplicado sob esta rua de Istanbul:
![Street](Results/Street_480_output.png)

### Boats (caso 4)
![Boats](ImagesAndAttribution/Boats_480_i.png)
Deseja-se remover estes barcos da imagem. O resultado gerado foi este:
![Boats](Results/Boats_480_output.png)

### Swan (caso 5)
![Swan](ImagesAndAttribution/Swan_480_i.png)
Deseja-se remover o reflexo na água. O resultado gerado foi esse:
![Swan](Results/Swan_480_output.png)

### Yosemite (caso 6)
![Yosemite](ImagesAndAttribution/Yosemite_480_i.png)
Deseja-se obter uma foto do parque natural sem a pessoa. O resultado foi esse:
![Yosemite](Results/Yosemite_480_output.png)

## 9. Conclusões
Neste projeto, foi implementado um algoritmo de Inpainting capaz de remover de imagens grandes objetos, restaurando-as. Foram utilizadas diversas técnicas para solução do problema, entre elas:
* __Onion Ring__: a imagem é restaurada de fora para dentro da máscara;
* __Root-Mean-Square Deviation (RMSD)__: usado para calcular semelhança entre patches;
* __Multi-Threading__: usado para diminuir o tempo de execução do algoritmo.
O maior desafio enfrentado ao longo do trabalho, de longe, foi o corte (slicing) das janelas e patches. Isso porque fazer tal exige uma longa análise e muito teste de mesa, afim de evitar imprecisão.

### Objetivos Alcançados
Praticamente todos os objetivos desejados a priori neste projeto foram alcançados. Foi implementado um algoritmo robusto, mas simples, e capaz de produzir diversos resultados para uma mesma imagem de entrada, sendo ao menos um deles aceitável. O algoritmo é capaz de trabalhar com imagens coloridas, diversos patches na máscara e de tamanho consideravelmente grande.

### Objetivos Falhos
A única falha do algoritmo se dá no parâmetro _tamanho de textura_ que o usuário deve fornecer. Não é trivial escolher um valor aceitável para este parâmetro, e em um cenário ideal, o usuário não deveria se prender a isto. Contudo, a escolha deste parâmetro é o ponto chave para determinar o resultado final. Inclusive, em alguns casos uma simples variação de uma (1) unidade é o bastante para afetar significamente a imagem gerada.

## 10. Referências
* Publicação [Object Removal by Exemplar-based Inpainting](https://www.microsoft.com/en-us/research/publication/object-removal-by-exemplar-based-inpainting/) de Junho de 2003.
* Vídeo-aula [DIP Lecture 23: Photomontage and inpainting](https://youtu.be/XTRO6yQOvJc?t=38m57s) de Maio de 2015.
* Material do professor [Moacir A. Ponti](https://github.com/maponti/).